In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Goal
- Develop a real estate pricing model that predicts house prices based on various features.
- Create an API that outputs the predicted price when given the features.

### Data Validation
- Was given three datasets: 'economics.csv', 'property_data.csv', and 'transactions.csv'


recipe: Numeric, unique identifier of recipe.
calories: Numeric, number of calories.
carbohydrate: Numeric, amount of carbohydrates in grams.
sugar: Numeric, amount of sugar in grams.
protein: Numeric, amount of protein in grams.
category: Character, type of recipe. Recipes are listed in one of ten possible groupings (Lunch/Snacks, Beverages, Potato, Vegetable, Meat, Chicken, Pork, Dessert, Breakfast, One Dish Meal).
servings: Numeric, number of servings for the recipe.
high_traffic: Character, if the traffic to the site was high when this recipe was shown, this is market with "High".

In [2]:
economics = pd.read_csv('Data/economics.csv')
transactions = pd.read_csv('Data/transactions.csv')
property = pd.read_csv('Data/property_data.csv')

#### Economics
- This dataset contains economics indicators, containing 1000 rows and 29 columns.
- It only contains numerical values.
- There aren't missing values.
- The dataset don't present outliers.
- I couldn't find any column that relate this dataset to the others and there is no time stamp column.
- I recommend to explore the source of this data to include the time stamp, so we can link this dataset to the others.

In [3]:
#Check for data type and missing values
economics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   interest_rate                    1000 non-null   float64
 1   unemployment_rate                1000 non-null   float64
 2   inflation_rate                   1000 non-null   float64
 3   gdp_growth_rate                  1000 non-null   float64
 4   labor_force_participation_rate   1000 non-null   float64
 5   consumer_confidence_index        1000 non-null   float64
 6   government_debt_to_gdp_ratio     1000 non-null   float64
 7   exchange_rate                    1000 non-null   float64
 8   stock_market_index               1000 non-null   float64
 9   housing_price_index              1000 non-null   float64
 10  manufacturing_pmi                1000 non-null   float64
 11  services_pmi                     1000 non-null   float64
 12  retail_sales_growth_r

In [4]:
#Check for outliers in economics
economics.describe()

,interest_rate,unemployment_rate,inflation_rate,gdp_growth_rate,labor_force_participation_rate,consumer_confidence_index,government_debt_to_gdp_ratio,exchange_rate,stock_market_index,housing_price_index,...,business_startups_rate,consumer_spending,investment_rate,savings_rate,tax_revenue,interest_coverage_ratio,bankruptcies_rate,business_closures_rate,consumer_debt_to_income_ratio,government_deficit_to_gdp_ratio
count,1000.00000,1000.000000,1000.000000,1000.000000,1000.00000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,5.09107,10.194420,0.028130,-0.112480,51.37444,99.34441,101.817990,1.522520,25149.785260,272.993190,...,49.462030,496943.352770,49.210700,49.428900,5.014713e+07,50.89443,5.072960,5.066430,245.593610,5.096980
std,2.80546,5.687702,2.861397,5.713069,29.48179,56.14199,58.087382,0.855760,14634.920801,133.009378,...,29.178894,289123.488684,28.538652,29.015521,2.917962e+07,29.38848,2.933028,2.856688,146.879964,2.915088
min,0.00000,0.020000,-4.990000,-9.980000,0.06000,0.05000,0.150000,0.005900,13.370000,50.130000,...,0.070000,565.310000,0.010000,0.120000,1.413380e+05,0.14000,0.010000,0.050000,0.410000,0.010000
25%,2.84750,5.355000,-2.482500,-5.020000,25.18500,51.43750,51.510000,0.847225,12428.930000,157.015000,...,23.647500,245601.067500,23.275000,23.522500,2.539032e+07,25.17250,2.647500,2.692500,112.082500,2.640000
50%,5.17000,10.485000,0.185000,-0.295000,52.80500,99.56000,100.990000,1.504700,25787.745000,271.950000,...,50.370000,501958.465000,50.100000,49.845000,4.944122e+07,50.69500,5.155000,5.105000,253.260000,5.195000
75%,7.44250,15.155000,2.442500,4.700000,76.68000,145.05750,152.450000,2.263350,37547.280000,387.502500,...,74.667500,733707.425000,74.010000,75.075000,7.625037e+07,76.92000,7.512500,7.520000,371.207500,7.625000
max,10.00000,19.970000,4.990000,9.950000,99.90000,199.96000,199.600000,2.998800,49977.020000,499.280000,...,99.830000,999429.530000,99.970000,99.980000,9.996664e+07,99.96000,10.000000,9.990000,497.060000,10.000000


### Transactions
- This dataset contains transactions history, containing 1000 rows and 10 columns
- 'transaction_date' column was in wrong format and was converted.
- 'buyer_name', 'seller_name', and 'agent_name' don't have enough variance in their categories, an unique person doesn't repeat between rows.
- The dataset doesn't present outliers

#### Data description

- **Property ID**: Numeric, unique identifier of the property sold.
- **Transaction Date**: Datetime, transaction date
- **Property Type**: Character, type of property. Categorize the property into: 'condo', 'apartment', 'house', and 'townhouse'.
- **Sale Price**: Numeric, price that the property was sold.
- **Purchase Price**: Numeric, price that the property was bought.
- **Comission Rate**: Numeric, comission that the agent charged for the transaction
- **Profit**: Numeric, difference between sale price and purchase price

In [5]:
#Check for data type and missing values
print(transactions.info())
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   property_id       1000 non-null   int64  
 1   transaction_date  1000 non-null   object 
 2   property_type     1000 non-null   object 
 3   sale_price        1000 non-null   float64
 4   purchase_price    1000 non-null   float64
 5   buyer_name        1000 non-null   object 
 6   seller_name       1000 non-null   object 
 7   agent_name        1000 non-null   object 
 8   commission_rate   1000 non-null   float64
 9   profit            1000 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 78.3+ KB
None


In [6]:
#Check for property type variance and unique values
print(transactions.groupby('property_type').size().sort_values(ascending=False))
transactions['property_type'].unique()

property_type
condo        259
house        259
apartment    245
townhouse    237
dtype: int64


array(['condo', 'apartment', 'house', 'townhouse'], dtype=object)

In [7]:
#Check for buyer_name variance and unique values
transactions.groupby('buyer_name').size().sort_values(ascending=False).head(5)

buyer_name
Zolly Eagle      1
Abbey Poon       1
Abdel Bucklee    1
Abel Buterton    1
Abel Speight     1
dtype: int64

In [8]:
#Check for seller_name variance and unique values
transactions.groupby('seller_name').size().sort_values(ascending=False).head(5)

seller_name
Zonnya McGahey      1
Abbie Giacobelli    1
Abbott Kleinfeld    1
Abigail Welham      1
Ade Coote           1
dtype: int64

In [9]:
#Check for agent_name variance and unique values
transactions.groupby('agent_name').size().sort_values(ascending=False).head(5)

agent_name
Zorine Hasney       1
Abagail Willwood    1
Abbe Laurenty       1
Abbie Lorain        1
Abner Glasard       1
dtype: int64

In [10]:
#Dropping columns without variance 
transactions = transactions.drop(['buyer_name', 'seller_name', 'agent_name'], axis=1)

In [11]:
#Check for outliers
transactions.describe()

,property_id,transaction_date,sale_price,purchase_price,commission_rate,profit
count,1000.000000,1000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,2020-12-22 03:51:50.400000,527628.918960,528515.232120,0.056590,-886.313160
min,1.000000,2020-01-01 00:00:00,51745.340000,51704.270000,0.010000,-916028.830000
25%,250.750000,2020-06-24 00:00:00,289499.695000,290190.015000,0.030000,-298189.092500
50%,500.500000,2020-12-21 00:00:00,520343.345000,534299.390000,0.060000,5592.625000
75%,750.250000,2021-06-18 00:00:00,756956.712500,774404.957500,0.080000,276438.325000
max,1000.000000,2021-12-29 00:00:00,999774.430000,998700.150000,0.100000,916765.210000
std,288.819436,NaN,273770.592932,278594.847411,0.026593,398046.225892


### Property
- This dataset contains property proprieties, containing 1000 rows and 15 columns
- 'property_id' and 'id' have the same values. I droped 'id'.
- 'location' has low variance. We have 1000 rows and 437 location types. The most frequent location has 8 observations. It's recommended to group these location by a more macro region.
- 'year_built' was in the wrong format, I conveted it to datetime.
- 'bedrooms' and 'num_bedrooms' have the same meaning. I droped 'num_bedrooms'
- 'num_bathrooms' was in the wrong format. I decided to drop this column, since we have the same information in 'bathroom' column.
- Since I found inconsistency into 'property' dataset and we have 'property_type' in both, property dataset and transaction dataset, I droped property_type in property dataset.

#### Data description

- **Location**: Character, location name (Street)
- **Size sqft**: Numeric, square feet size of the property
- **Bedrooms**: Numeric, number of bedrooms.
- **Property ID**: Numeric, unique identifier of the property.
- **Property Size**: Numeric, size of the property
- **Year Built**: Numeric, year in that the build was constructed.
- **Listing Price**: Numeric, price that the property was anouced.
- **Neighborhood**: Character, property neighborhood.
- **Garage Spaces**: Numeric, number of garage spaces
- **Has Pool**: Boolean, if the property has pool or not.

In [24]:
property.property_size

0      4157
1      2393
2      3735
3      4427
4      3328
       ... 
995    2529
996    4030
997    3995
998     863
999    4461
Name: property_size, Length: 1000, dtype: int64

In [12]:
print(property.info())
property['year_built'] = pd.to_datetime(property['year_built'], format='%Y')
property = property.drop(['num_bathrooms', 'num_bedrooms', 'property_type'], axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1000 non-null   int64  
 1   location       1000 non-null   object 
 2   size_sqft      1000 non-null   int64  
 3   bedrooms       1000 non-null   int64  
 4   bathrooms      1000 non-null   int64  
 5   property_id    1000 non-null   int64  
 6   property_type  1000 non-null   object 
 7   property_size  1000 non-null   int64  
 8   year_built     1000 non-null   int64  
 9   listing_price  1000 non-null   float64
 10  num_bedrooms   1000 non-null   int64  
 11  num_bathrooms  1000 non-null   float64
 12  neighborhood   1000 non-null   object 
 13  garage_spaces  1000 non-null   int64  
 14  has_pool       1000 non-null   bool   
dtypes: bool(1), float64(2), int64(9), object(3)
memory usage: 110.5+ KB
None


In [13]:
#Check for property id and property_id. Both have the same values, we can drop one of those columns
(property['id'] - property['property_id']).value_counts()
property = property.drop('id', axis=1)

In [14]:
#Check for location variance and unique values
property.groupby('location').size().sort_values(ascending=False)

location
Evergreen    8
Arapahoe     6
Union        6
Barnett      6
Basil        6
            ..
Waxwing      1
Waubesa      1
Warner       1
Warbler      1
2nd          1
Length: 437, dtype: int64

In [15]:
#Check for neighborhood variance and unique values
property.groupby('neighborhood').size().sort_values(ascending=False)

neighborhood
rural       352
suburban    327
urban       321
dtype: int64

In [16]:
#Check garage_spaces
property['garage_spaces'].value_counts()

garage_spaces
0    264
2    254
1    253
3    229
Name: count, dtype: int64

In [17]:
property['bedrooms'].value_counts()

bedrooms
4    221
1    211
3    199
2    192
5    177
Name: count, dtype: int64

In [18]:
property['bathrooms'].value_counts()

bathrooms
4    179
1    170
3    168
5    168
6    162
2    153
Name: count, dtype: int64

In [19]:
#Check for numerical values
property.describe()

,size_sqft,bedrooms,bathrooms,property_id,property_size,year_built,listing_price,garage_spaces
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000,1000.000000,1000.000000
mean,501.028000,2.961000,3.508000,500.500000,2760.230000,1962-10-22 15:21:36,544311.871870,1.448000
min,1.000000,1.000000,1.000000,1.000000,505.000000,1900-01-01 00:00:00,100358.270000,0.000000
25%,260.750000,2.000000,2.000000,250.750000,1579.500000,1933-10-01 18:00:00,320294.705000,0.000000
50%,480.500000,3.000000,4.000000,500.500000,2792.500000,1963-01-01 00:00:00,537369.045000,1.000000
75%,750.000000,4.000000,5.000000,750.250000,3944.000000,1994-04-02 06:00:00,764595.350000,2.000000
max,1000.000000,5.000000,6.000000,1000.000000,4993.000000,2022-01-01 00:00:00,999759.420000,3.000000
std,287.186088,1.401943,1.699065,288.819436,1319.579978,NaN,258375.306032,1.111094


In [20]:
property.head(5)

,location,size_sqft,bedrooms,bathrooms,property_id,property_size,year_built,listing_price,neighborhood,garage_spaces,has_pool
0,Kinsman,768,3,2,1,4157,2012-01-01,866818.73,urban,1,False
1,Forest Run,794,1,6,2,2393,1997-01-01,393604.20,rural,0,True
2,Lakeland,385,3,5,3,3735,2007-01-01,499365.09,urban,2,True
3,Drewry,147,5,5,4,4427,1941-01-01,495378.90,suburban,0,True
4,Jenna,156,3,5,5,3328,1942-01-01,454512.91,urban,0,False
